In [5]:
infile = open('day18_input.txt', 'r')
data_lines = []
for line in infile:
    data_lines.append(line[:-1])

infile.close()

In [39]:
import random

class vec2i(object):
    def __init__(self, x, y):
        self.x = int(round(x))
        self.y = int(round(y))

    @classmethod
    def from_index(cls, ind):
        ssplit = ind[1:].split('P')
        yoff = 1
        if(len(ssplit) == 1):
            ssplit = ind[1:].split('N')
            yoff = -1
        
        xoff = 1
        if(ind[0] == 'N'):
            xoff = -1
        
        return cls(xoff * int(ssplit[0]), yoff * int(ssplit[1]))
        
    def index(self):
        xpart = ""
        if(self.x < 0):
            xpart = "N%i" % abs(self.x) 
        else:
            xpart = "P%i" % abs(self.x)
        ypart = ""
        if(self.y < 0):
            ypart = "N%i" % abs(self.y) 
        else:
            ypart = "P%i" % abs(self.y)
        return "%s%s" % (xpart,ypart)
    
    def step(self, v2):
        return vec2i(self.x + v2.x, self.y + v2.y)
    
    def diff(self, v2):
        return vec2i(v2.x - self.x, v2.y - self.y)
    
    def copy(self):
        return vec2i(self.x, self.y)
    
    def __str__(self):
        return ("(%i,%i)" % (self.x, self.y))
    
    def __repr__(self):
        return ("(%i,%i)" % (self.x, self.y))

def rev_dir(forward):
    if(forward == 1):
        return 2
    elif(forward == 2):
        return 1
    elif(forward == 3):
        return 4
    elif(forward == 4):
        return 3
    
    
    
def move_steps(idir):
    return {1:vec2i(0,-1), 2:vec2i(0,1), 3:vec2i(-1,0), 4:vec2i(1,0)}[idir]
    
class Intersection(object):
    def __init__(self, pos, char):
        self.pos = pos
        self.paths = {} #direction -> path
        self.pathstart = {} #whether this intersection is the start of given path or end
        self.char = char
        
    def add_path(self, idir, path):
        self.paths[idir] = path
        self.pathstart[idir] = (path.start == self)
        
    def is_lock(self):
        ai = ord(self.char)
        return ai >= ord('A') and ai <= ord("Z")
    
    def is_key(self):
        ai = ord(self.char)
        return ai >= ord('a') and ai <= ord("z")
        
    def get_routes(self):
        routes = {}
        for idir in self.paths:
            path = self.paths[idir]
            if path.start != self:
                routes[path.start.pos.index()] = path.length + 1
            elif path.end != self:
                routes[path.end.pos.index()] = path.length + 1
        return routes
        

class Path(object):
    def __init__(self, start, firststep, firstdir, current_endpt):
        self.steps = [firststep]
        self.stepdirs = [] #first dir is steps[0] -> steps[1] (since intersection includes 
                           #direction), last is steps[n] -> end_intersection
        self.firstdir = firstdir
        self.length = 1
        self.start = start #intersection
        self.end = None
        self.current_endpt = current_endpt
        self.is_loop = False #when a path starts and ends at the same intersection
        self.dead_end = False
        
    def add_step(self, step, stepdir):
        self.steps.append(step)
        self.stepdirs.append(stepdir)
        self.length = self.length + 1
            
    def end_path(self, end, idir):
        
        #case of a "room", ie two adjacent intersections
        if self.steps[0] == end:
            self.steps = []
            self.stepdirs = []
            self.length = 0
        self.end = end
        self.stepdirs.append(idir)
        self.is_loop = (self.end == self.start)
        self.end.add_path(rev_dir(idir), self)
        
    def mark_dead_end(self):
        self.dead_end = True

        
class Flooder(object):
    def __init__(self):
        self.openlst = []
        self.visited = set()
        self.exploded_intersections = set()
        
    def pop(self):
        path = self.openlst[0]
        self.openlst = self.openlst[1:]
        return path
    
    def nonempty(self):
        return len(self.openlst) > 0
    
class StepInfo(object):
    def __init__(self, ind, pos, idir):
        self.ind = ind
        self.pos = pos
        self.idir = idir
        
class Area(object):
    def __init__(self, init_data):
        self.map = {}
        self.width = len(init_data[0])
        self.height = len(init_data)
        self.data = init_data
        self.intersections = {} #pos to intersection
        self.paths = {} #pos to path
        self.entrance = None

        entrancepos = None
        self.floorspaces = 0
        for y in range(1, self.height - 1):
            for x in range(1, self.width - 1):
                char = self.data[y][x]
                if char == '#':
                    continue
                self.floorspaces = self.floorspaces + 1
                ind = vec2i(x,y).index()
                if char == '@':
                    self.entrance = ind
                    entrancepos = vec2i(x,y)
                outlets = 0
                if(self.data[y-1][x] != '#'):
                    outlets = outlets + 1
                if(self.data[y+1][x] != '#'):
                    outlets = outlets + 1
                if(self.data[y][x-1] != '#'):
                    outlets = outlets + 1
                if(self.data[y][x+1] != '#'):
                    outlets = outlets + 1
                if(outlets > 2):
                    self.intersections[ind] = Intersection(ind, char)
                else:
                    #coerce all the keys and doors to be intersections, so we don't have to think about mid-path routing
                    ai = ord(char)
                    if((ai >= ord('A') and ai <= ord("Z")) or 
                       (ai >= ord('a') and ai <= ord("z")) ):
                        self.intersections[ind] = Intersection(ind, char)
        
        #special case the entrance, if it's not a real intersection, make it one anyway
        if(not(self.entrance in self.intersections)):
            char = self.data[entrancepos.y][entrancepos.x]
            self.intersections[self.entrance] = Intersection(self.entrance, char)
            
        #todo flood, create paths. once a path has started, follow it until an intersection is found
        flood = Flooder()
        flood.visited.add(self.entrance)
        self.explode(entrancepos, flood)
        
        while(flood.nonempty()):
            cur_path = flood.pop()
            
            startpos = cur_path.current_endpt
            startind = startpos.index()
            pathing = True
            
            if(startind in self.intersections):
                if len(cur_path.stepdirs) > 0 :
                    idir = cur_path.stepdirs[-1]
                else:
                    idir = cur_path.firstdir
                cur_path.end_path(self.intersections[startind], idir)
                self.explode(startpos, flood)
                pathing = False
            
            while(pathing):
                newind = None
                newpos = None
                newdir = None
                
                spots = self.open_spots(startpos, flood)
                if len(spots) > 1:
                    print("error, found multiple indexes in non-intersection %" % dirind)
                
                if len(spots) > 0:
                    newind = spots[0].ind
                    newpos = spots[0].pos
                    newdir = spots[0].idir

                #this should only occur in the case of a loop (including the null case of a single step dead end)
                if newind == None:
                    cur_path.mark_dead_end()
                    pathing = False
                elif newind in self.intersections:
                    cur_path.end_path(self.intersections[newind], newdir)
                    self.explode(newpos, flood)
                    pathing = False
                else:
                    cur_path.add_step(newind, newdir)
                    startpos = newpos
                    startind = newind

                flood.visited.add(newind)
    
    def explode(self, pos, flood):
        posind = pos.index()
        if(not(posind in flood.exploded_intersections)):
            flood.exploded_intersections.add(posind)
            for spot in self.open_spots(pos, flood):
                flood.visited.add(spot.ind)
                newpath = Path(self.intersections[posind], spot.ind, spot.idir, spot.pos)
                self.paths[spot.ind] = newpath 
                self.intersections[posind].add_path(spot.idir, newpath)
                flood.openlst.append(newpath)
                
    def open_spots(self, pos, flood):
        results = []
        for idir in range(1,5):
            testpos = pos.step(move_steps(idir))
            if(self.data[testpos.y][testpos.x] == '#'):
                continue
            dirind = testpos.index()
            if dirind in flood.visited:
                continue
            results.append(StepInfo(dirind, testpos, idir))
        return results
            
            
            

In [40]:
area = Area(data_lines)

In [41]:
print ('# paths: %i\n# intersections: %i\n# floor spaces: %i' % (len(area.paths), len(area.intersections), area.floorspaces))

# paths: 397
# intersections: 224
# floor spaces: 3201


In [42]:
(area.floorspaces - len(area.intersections)) / len(area.paths)




7.498740554156171